In [1]:
from gtda.homology import EuclideanCechPersistence,VietorisRipsPersistence
#from gtda.diagrams import PersistenceImage
from gtda.pipeline import Pipeline
from gtda.diagrams import Filtering

In [2]:
import tensorflow as tf
import gudhi as gd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils
import scipy.spatial as spatial
import json
from rich import print
from random import choice, sample
from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from gc import collect
from pickle import load

with open("9701_cech_persistence_entropy_012.pickle", mode = "rb") as data:
    save_dict = load(data)
    train_molecules = save_dict["train_molecules"]
    train_data = save_dict["train_data"]
    
    test_molecules = save_dict["test_molecules"]
    test_data = save_dict["test_data"]
    
%matplotlib inline
del save_dict
collect()

4

In [3]:
print(train_data.shape, test_data.shape)

(7761, 9)
(1940, 9)

# Extract descriptors
(this can be done from rdkit or from the descriptor .csv)

In [4]:
df = pd.read_csv("data/Drugbank_some_descriptors.csv")
cntr = 0
for d in df:
    cntr += 1
print(cntr)

126

### Filter descriptors which are unavailable

In [5]:
descriptors = list()

# consider only descriptors which have few missing values and which are floating point
for d in df:
    if df[d].isna().sum() < 800 and df[d].dtype == np.float64:
        descriptors.append(d)

num_descriptors = len(descriptors)
print(descriptors)

# select only smiles which have these descriptor values
train_smiles = set(train_molecules)
test_smiles = set(test_molecules)

for i,d in enumerate(descriptors):
    avail_rows = df[np.logical_not(df[d].isna())]
    train_smiles = set(avail_rows["SMILES"]).intersection(train_smiles)
    test_smiles = set(avail_rows["SMILES"]).intersection(test_smiles)

train_idxs = np.asarray(sorted([ train_molecules.index(t) for t in train_smiles ]))
test_idxs = np.asarray(sorted([ test_molecules.index(t) for t in test_smiles ]))

train_molecules = np.asarray(train_molecules)
train_molecules = train_molecules[train_idxs]
train_data = train_data[train_idxs]

test_molecules = np.asarray(test_molecules)
test_molecules = test_molecules[test_idxs]
test_data = test_data[test_idxs]

[
    'ACD Descriptors;logP',
    'ACD logD-logP;logP',
    'MOE Descriptors;h_ema',
    'MOE Descriptors;h_emd',
    'MOE Descriptors;h_emd_c',
    'MOE Descriptors;h_logp',
    'MOE Descriptors;h_logs',
    'MOE Descriptors;h_log_dbo',
    'MOE Descriptors;h_log_pbo',
    'MOE Descriptors;h_mr',
    'OEselma Descriptors;bromine_count',
    'OEselma Descriptors;carbon_count',
    'OEselma Descriptors;clorine_count',
    'OEselma Descriptors;ertl_tpsa',
    'OEselma Descriptors;fluorine_count',
    'OEselma Descriptors;hba',
    'OEselma Descriptors;hba_lipinski',
    'OEselma Descriptors;hbd',
    'OEselma Descriptors;hbd_lipinski',
    'OEselma Descriptors;halogen_count',
    'OEselma Descriptors;iodine_count',
    'OEselma Descriptors;max_flex_chain_1',
    'OEselma Descriptors;max_flex_chain_2',
    'OEselma Descriptors;max_flex_chain_3',
    'OEselma Descriptors;max_rigid_chain',
    'OEselma Descriptors;mol_volume_2d',
    'OEselma Descriptors;mw',
    'OEselma Descriptors;npsa',
    'OEselma Descriptors;neg_ioniz',
    'OEselma Descriptors;nitrogen_count',
    'OEselma Descriptors;nonpolar_count',
    'OEselma Descriptors;nonpolar_count_per_mw',
    'OEselma Descriptors;rigid_frag_count',
    'OEselma Descriptors;aromatic_ring_count',
    'OEselma Descriptors;atom_count',
    'OEselma Descriptors;bond_count',
    'OEselma Descriptors;rigid_bond_count',
    'OEselma Descriptors;ring_count',
    'OEselma Descriptors;rot_bond_count',
    'OEselma Descriptors;oxygen_count',
    'OEselma Descriptors;psa',
    'OEselma Descriptors;phosphorous_count',
    'OEselma Descriptors;polar_count',
    'OEselma Descriptors;polar_count_per_mw',
    'OEselma Descriptors;pos_ioniz',
    'OEselma Descriptors;silicon_count',
    'OEselma Descriptors;sulphur_count',
    'OEselma Descriptors;tsa',
    'Ertl TPSA;Ertl TPSA',
    'Exact Mass;Exact Mass',
    'Heavy Atom Count;Heavy Atom Count',
    'Molecular Volume (2D);Molecular Volume (2D)',
    'Molecular Weight;Molecular Weight',
    'NPSA;NPSA',
    'PSA;PSA',
    'Ring Count;Ring Count',
    'Rotatable Bond Count;Rotatable Bond Count',
    'AZlogD74;prediction',
    'AZlogD74;sd_prediction',
    'ChromlogD;prediction',
    'ChromlogD;sd_prediction',
    'ALogP',
    'Mol Weight',
    'Molar Refractivity',
    'Polarizability',
    'TPSA',
    'Solubility DD Class;confidence',
    'Solubility DD Class;credibility',
    'SCScore;SCScore',
    'AZlogD74 (NN);nearest_neighbours[]- GT similarity',
    'AZlogD74 (NN);nearest_neighbours[]- GT measured',
    'clogP;clogp',
    'clogP;clogp_error',
    'OEselma Descriptors;part_flex_chain'
]

In [6]:
print(train_data.shape, test_data.shape)

(7205, 9)
(1787, 9)

### Build numpy arrays of descriptors

In [7]:
collect()
train_labels = np.empty((train_data.shape[0], num_descriptors))
test_labels = np.empty((test_data.shape[0], num_descriptors))
train_labels[:] = np.nan
test_labels[:] = np.nan

for i,mol in enumerate(tqdm(train_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    train_labels[i,:] = np.asarray(data_vec)

for i,mol in enumerate(tqdm(test_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    test_labels[i,:] = np.asarray(data_vec)

  0%|          | 0/7205 [00:00<?, ?it/s]

  0%|          | 0/1787 [00:00<?, ?it/s]

In [8]:
print(train_labels.shape, test_labels.shape)

(7205, 74)
(1787, 74)

In [9]:
print(not np.isnan(train_labels).any(), not np.isnan(test_labels).any())

True True

In [10]:
with np.printoptions(2, suppress = True):
    print(choice(train_labels))

[ -1.46  -1.46   6.36   9.38   3.85   0.     3.63   0.9   -1.75   0.
   0.     6.     0.    90.15   0.     5.     5.     4.     4.     0.
   0.     1.     0.     0.     5.   258.21 164.2   91.06   0.     0.
   1.     0.01   1.     0.    11.    11.     4.     1.     3.     5.
  99.03   0.     5.     0.03   0.     0.     0.   190.    90.15 164.07
  11.   258.21 164.2   91.06  99.03   1.     3.    -1.41   0.15  -2.27
   0.2   -1.34 164.16  34.83  14.2   90.15   0.93   0.39   1.99   0.55
   4.28  -2.45   0.     1.  ]

# Setup OLS model

In [11]:
from sklearn import linear_model

#creating a regression object
reg = linear_model.LinearRegression()

#runnin OLS on your data, assuming that you already have arrays x and y
reg.fit( train_data, train_labels )
reg.score(train_data, train_labels)

0.21731159396226127

In [15]:
reg.coef_.shape

(74, 9)

In [12]:
# Using 'auto'/'sum_over_batch_size' reduction type.
mae = tf.keras.losses.MeanAbsoluteError()
mae(test_labels, reg.predict(test_data)).numpy()

2022-01-09 18:09:52.861684: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-09 18:09:53.245526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6792 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


19.298789978027344

In [13]:
predicted = reg.predict(test_data)

out_dict = dict()
out_dict["Descriptor"] = list()
out_dict["Average"] = list()

for d in descriptors:
    out_dict["Descriptor"].append(d)
    out_dict["Average"].append(df[d].mean(skipna = True)) # population mean for this descriptor

out_dict["Test Average"] = test_labels.mean(axis = 0)
out_dict["MAE"] = np.abs(test_labels-predicted).mean(axis = 0)
Q = out_dict["MAE"]/(np.abs(out_dict["Average"]) + out_dict["MAE"]) # good if close to 0
out_dict["Score"] = 1-Q

pd.set_option("display.max_rows", None, "display.max_columns", None)
out_df = pd.DataFrame(data = out_dict)
print(out_dict["MAE"].sum()/74) # THIS IS WHAT tf.keras.losses.MeanAbsoluteError() DOES
out_df = out_df.sort_values("Score")
out_df
#print(out_df.to_latex())

19.298790657527803

,Descriptor,Average,Test Average,MAE,Score
57,AZlogD74;prediction,0.608286,0.807755,2.214187,0.215515
59,ChromlogD;prediction,1.046476,1.259415,2.635566,0.284211
20,OEselma Descriptors;iodine_count,0.028450,0.039731,0.068471,0.293539
10,OEselma Descriptors;bromine_count,0.034856,0.033016,0.067484,0.340591
14,OEselma Descriptors;fluorine_count,0.276967,0.304421,0.500981,0.356022
41,OEselma Descriptors;phosphorous_count,0.122751,0.123111,0.221552,0.356520
12,OEselma Descriptors;clorine_count,0.168629,0.182988,0.301226,0.358896
9,MOE Descriptors;h_mr,0.327136,0.322575,0.508297,0.391577
46,OEselma Descriptors;sulphur_count,0.289119,0.282037,0.434317,0.399647
19,OEselma Descriptors;halogen_count,0.508902,0.560157,0.743283,0.406412


In [14]:
print(out_df.to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &                                         Descriptor &     Average &  Test Average &      
MAE &     Score \\
\midrule
57 &                                AZlogD74;prediction &    0.608286 &      0.807755 &    
2.214187 &  0.215515 \\
59 &                               ChromlogD;prediction &    1.046476 &      1.259415 &    
2.635566 &  0.284211 \\
20 &                   OEselma Descriptors;iodine\_count &    0.028450 &      0.039731 &    
0.068471 &  0.293539 \\
10 &                  OEselma Descriptors;bromine\_count &    0.034856 &      0.033016 &    
0.067484 &  0.340591 \\
14 &                 OEselma Descriptors;fluorine\_count &    0.276967 &      0.304421 &    
0.500981 &  0.356022 \\
41 &              OEselma Descriptors;phosphorous\_count &    0.122751 &      0.123111 &    
0.221552 &  0.356520 \\
12 &                  OEselma Descriptors;clorine\_count &    0.168629 &      0.182988 &    
0.301226 &  0.358896 \\
9  &                               MOE Descriptors;h\_mr &    0.327136 &      0.322575 &    
0.508297 &  0.391577 \\
46 &                  OEselma Descriptors;sulphur\_count &    0.289119 &      0.282037 &    
0.434317 &  0.399647 \\
19 &                  OEselma Descriptors;halogen\_count &    0.508902 &      0.560157 &    
0.743283 &  0.406412 \\
71 &                                        clogP;clogp &    1.655366 &      1.896323 &    
2.360496 &  0.412207 \\
28 &                      OEselma Descriptors;neg\_ioniz &    0.539802 &      0.452714 &    
0.641812 &  0.456834 \\
8  &                          MOE Descriptors;h\_log\_pbo &    1.932621 &      2.141476 &    
2.082669 &  0.481315 \\
1  &                                 ACD logD-logP;logP &    2.053207 &      2.073000 &    
2.170762 &  0.486085 \\
0  &                               ACD Descriptors;logP &    2.055326 &      2.073000 &    
2.170762 &  0.486343 \\
72 &                                  clogP;clogp\_error &   20.365517 &     17.612759 &   
21.111785 &  0.491004 \\
61 &                                              ALogP &    2.202912 &      2.420230 &    
1.940984 &  0.531604 \\
44 &                      OEselma Descriptors;pos\_ioniz &    1.011211 &      1.043089 &    
0.820734 &  0.551988 \\
23 &               OEselma Descriptors;max\_flex\_chain\_3 &    1.148954 &      1.130946 &   
0.931127 &  0.552360 \\
70 &    AZlogD74 (NN);nearest\_neighbours[]- GT measured &    1.637720 &      1.663844 &    
1.307334 &  0.556092 \\
17 &                            OEselma Descriptors;hbd &    2.674517 &      2.675993 &    
1.900881 &  0.584543 \\
18 &                   OEselma Descriptors;hbd\_lipinski &    2.940744 &      2.945719 &    
1.962018 &  0.599814 \\
22 &               OEselma Descriptors;max\_flex\_chain\_2 &    1.691916 &      1.711807 &   
1.115499 &  0.602660 \\
29 &                 OEselma Descriptors;nitrogen\_count &    2.583797 &      2.632345 &    
1.690824 &  0.604450 \\
33 &            OEselma Descriptors;aromatic\_ring\_count &    1.607348 &      1.691662 &    
0.998422 &  0.616842 \\
39 &                   OEselma Descriptors;oxygen\_count &    4.206029 &      4.003358 &    
2.338538 &  0.642675 \\
45 &                  OEselma Descriptors;silicon\_count &    0.004239 &      0.000560 &    
0.002336 &  0.644773 \\
21 &               OEselma Descriptors;max\_flex\_chain\_1 &    2.943565 &      3.026861 &   
1.517344 &  0.659858 \\
58 &                             AZlogD74;sd\_prediction &    0.729417 &      0.712565 &    
0.371994 &  0.662257 \\
42 &                    OEselma Descriptors;polar\_count &    6.059067 &      5.834359 &    
2.927670 &  0.674223 \\
60 &                            ChromlogD;sd\_prediction &    0.806016 &      0.787190 &    
0.389351 &  0.674283 \\
30 &                 OEselma Descriptors;nonpolar\_count &   12.086764 &     12.627308 &    
5.805089 &  0.675546 \\
40 &                            OEselma Descriptors;psa &  106.494461 &    102.077127 &   
50.681520 & 